# Part I. ETL Pipeline for Pre-Processing the Files

##  FOLLOWING CODE  IS USED FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/shilpamadini/Desktop/DEND/sparkify-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create  Apache Cassandra key space and the respective tables. 

## The <font color=red>event_datafile_new.csv</font>  contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data in the file appeara like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

##  Apache Cassandra code

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance in your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Best strategy for fast reads in appache cassandra is to model the database tables on the queries you want to run.

## The following are the three questions we want to ask of the data.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# For this query session_id and item_in_session are considered the composite key as the combination of the both those
# values presents uniqueness to the data. since we are also partitioning the data with this composite key data 
# selection on these keys becomes fast.

query = "CREATE TABLE IF NOT EXISTS musicapp_session_history "
query = query + "(session_id int, item_in_session int, artist_name text, title text, length float, \
                   PRIMARY KEY (session_id,item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [14]:
# load required fields from data file to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO musicapp_session_history (session_id, item_in_session, artist_name,title,length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

####  verify that the data is inserted into each table

In [12]:
## SELECT statement to verify the data was entered into the table
query = "select artist_name,title,length from musicapp_session_history WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# For this query user_id and session_id are composite key. we chose item_id as the clustering column as we need to sort the 
# song sorted by item_in_session.

query = "CREATE TABLE IF NOT EXISTS musicapp_user_history "
query = query + "(user_id int, session_id int, item_in_session int,artist_name text, title text, first_name text, \
                   last_name text, PRIMARY KEY ((user_id, session_id),item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [15]:
# load required fields from data file to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO musicapp_user_history (user_id, session_id, item_in_session, artist_name, title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]),line[0],line[9],line[1],line[4]))

In [16]:
## SELECT statement to verify the data was entered into the table
query = "select artist_name,title,first_name,last_name from musicapp_user_history WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [20]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# For this query we chose Primary key as the composite key of title,first_name,last_name and user_id as it presents
# uniqueness to the data. 
query = "CREATE TABLE IF NOT EXISTS musicapp_song_history "
query = query + "(title text, first_name text, last_name text, user_id int, \
                  PRIMARY KEY (title, first_name, last_name,user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [22]:
# load required fields from data file to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO musicapp_song_history (title, first_name, last_name,user_id)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9],line[1],line[4],int(line[10])))

In [24]:
## SELECT statement to verify the data was entered into the table
query = "select first_name, last_name , user_id from musicapp_song_history WHERE title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name,  row.last_name, row.user_id)

Jacqueline Lynch 29
Sara Johnson 95
Tegan Levine 80


### Drop the tables before closing out the sessions

In [25]:
## Drop the table before closing out the sessions

In [26]:
query = "drop table musicapp_session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table musicapp_user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table musicapp_song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()